In [1]:
!pip install pandas scikit-learn -q

In [2]:
import json
import random
import pandas as pd
from sklearn.model_selection import train_test_split
import os

random.seed(42)
os.makedirs('dataset', exist_ok=True)
os.makedirs('dataset/by_operation', exist_ok=True)
print("✅ Setup complete")

✅ Setup complete


In [3]:
SCHEMAS = {
    "employees": {
        "fields": ["employee_id", "name", "department", "salary", "hire_date", "email", "manager_id", "status"],
        "dtypes": {"employee_id": "INT", "name": "VARCHAR(100)", "department": "VARCHAR(50)",
                   "salary": "DECIMAL(10,2)", "hire_date": "DATE", "email": "VARCHAR(100)",
                   "manager_id": "INT", "status": "VARCHAR(20)"}
    },
    "products": {
        "fields": ["product_id", "name", "category", "price", "stock", "supplier_id", "created_at"],
        "dtypes": {"product_id": "INT", "name": "VARCHAR(100)", "category": "VARCHAR(50)",
                   "price": "DECIMAL(10,2)", "stock": "INT", "supplier_id": "INT", "created_at": "DATETIME"}
    },
    "orders": {
        "fields": ["order_id", "customer_id", "product_id", "quantity", "total_amount", "order_date", "status"],
        "dtypes": {"order_id": "INT", "customer_id": "INT", "product_id": "INT",
                   "quantity": "INT", "total_amount": "DECIMAL(10,2)", "order_date": "DATE", "status": "VARCHAR(20)"}
    },
    "customers": {
        "fields": ["customer_id", "name", "email", "phone", "city", "country", "created_at"],
        "dtypes": {"customer_id": "INT", "name": "VARCHAR(100)", "email": "VARCHAR(100)",
                   "phone": "VARCHAR(20)", "city": "VARCHAR(50)", "country": "VARCHAR(50)", "created_at": "DATETIME"}
    },
    "departments": {
        "fields": ["dept_id", "dept_name", "location", "budget", "manager_id"],
        "dtypes": {"dept_id": "INT", "dept_name": "VARCHAR(50)", "location": "VARCHAR(100)",
                   "budget": "DECIMAL(15,2)", "manager_id": "INT"}
    },
    "inventory": {
        "fields": ["item_id", "product_id", "warehouse", "quantity", "last_updated"],
        "dtypes": {"item_id": "INT", "product_id": "INT", "warehouse": "VARCHAR(50)",
                   "quantity": "INT", "last_updated": "DATETIME"}
    }
}

TABLE_NAMES  = list(SCHEMAS.keys())
DEPARTMENTS  = ["HR", "Engineering", "Sales", "Marketing", "Finance", "Operations", "IT", "Legal"]
CITIES       = ["New York", "London", "Mumbai", "Berlin", "Tokyo", "Sydney", "Paris", "Dubai"]
STATUSES     = ["active", "inactive", "pending", "completed", "cancelled"]
CATEGORIES   = ["Electronics", "Furniture", "Clothing", "Food", "Machinery", "Software"]
DB_NAMES     = ["company_db", "sales_db", "hr_db", "inventory_db", "crm_db"]

print("✅ Schemas loaded:", TABLE_NAMES)

✅ Schemas loaded: ['employees', 'products', 'orders', 'customers', 'departments', 'inventory']


In [4]:
def rnd_table():
    return random.choice(TABLE_NAMES)

def rnd_fields(table, n=None):
    fields = SCHEMAS[table]["fields"]
    n = n or random.randint(2, len(fields))
    return random.sample(fields, min(n, len(fields)))

def rnd_field(table):
    return random.choice(SCHEMAS[table]["fields"])

def rnd_val(field):
    if "id"       in field: return random.randint(1, 999)
    if "salary"   in field: return round(random.uniform(30000, 150000), 2)
    if "price"    in field: return round(random.uniform(10, 5000), 2)
    if "amount"   in field: return round(random.uniform(100, 50000), 2)
    if "quantity" in field or "stock" in field: return random.randint(1, 500)
    if "date"     in field: return random.choice(["2024-01-15", "2023-06-30", "2024-12-01", "2022-03-22"])
    if "status"   in field: return random.choice(STATUSES)
    if "department" in field: return random.choice(DEPARTMENTS)
    if "city"     in field: return random.choice(CITIES)
    if "category" in field: return random.choice(CATEGORIES)
    if "email"    in field: return "user@example.com"
    return f"sample_{field}"

def quote(val):
    if isinstance(val, str): return f"'{val}'"
    return str(val)

print("✅ Helpers ready")

✅ Helpers ready


In [5]:
def sql_create(_):
    t = rnd_table()
    fields = SCHEMAS[t]["fields"]
    dtypes = SCHEMAS[t]["dtypes"]
    col_defs = ", ".join([f"{f} {dtypes[f]}" for f in fields])
    nl_templates = [
        f"Create a table called {t} with columns {', '.join(fields)}",
        f"Make a new {t} table storing {', '.join(fields[:3])} and other details",
        f"Set up the {t} table in the database",
        f"I want to create a {t} table with all its fields",
        f"Build a table for {t} data"
    ]
    sql = f"CREATE TABLE {t} ({col_defs});"
    return random.choice(nl_templates), sql


def sql_read(_):
    t = rnd_table()
    fields = rnd_fields(t)
    f = rnd_field(t)
    val = rnd_val(f)
    cond_templates = [
        (f"Get {', '.join(fields)} from {t} where {f} is {val}",
         f"SELECT {', '.join(fields)} FROM {t} WHERE {f} = {quote(val)};"),
        (f"Show all records from {t}",
         f"SELECT * FROM {t};"),
        (f"Find {t} records where {f} equals {val}",
         f"SELECT * FROM {t} WHERE {f} = {quote(val)};"),
        (f"Retrieve the top 10 rows from {t} ordered by {f}",
         f"SELECT * FROM {t} ORDER BY {f} LIMIT 10;"),
        (f"Count total {t} records",
         f"SELECT COUNT(*) FROM {t};"),
        (f"List distinct {f} values from {t}",
         f"SELECT DISTINCT {f} FROM {t};")
    ]
    return random.choice(cond_templates)


def sql_update(_):
    t = rnd_table()
    set_f = rnd_field(t)
    set_v = rnd_val(set_f)
    where_f = "employee_id" if "employee_id" in SCHEMAS[t]["fields"] else rnd_field(t)
    where_v = rnd_val(where_f)
    templates = [
        (f"Update {set_f} to {set_v} for {t} where {where_f} is {where_v}",
         f"UPDATE {t} SET {set_f} = {quote(set_v)} WHERE {where_f} = {quote(where_v)};"),
        (f"Change the {set_f} of {t} record {where_v} to {set_v}",
         f"UPDATE {t} SET {set_f} = {quote(set_v)} WHERE {where_f} = {quote(where_v)};"),
        (f"Modify {set_f} in {t} table where {where_f} = {where_v}",
         f"UPDATE {t} SET {set_f} = {quote(set_v)} WHERE {where_f} = {quote(where_v)};")
    ]
    return random.choice(templates)


def sql_delete(_):
    t = rnd_table()
    f = rnd_field(t)
    v = rnd_val(f)
    templates = [
        (f"Delete records from {t} where {f} is {v}",
         f"DELETE FROM {t} WHERE {f} = {quote(v)};"),
        (f"Remove all {t} entries where {f} equals {v}",
         f"DELETE FROM {t} WHERE {f} = {quote(v)};"),
        (f"Erase {t} rows with {f} = {v}",
         f"DELETE FROM {t} WHERE {f} = {quote(v)};")
    ]
    return random.choice(templates)


def sql_join(_):
    t1 = "employees"
    t2 = random.choice(["departments", "orders"])
    join_col = "manager_id" if t2 == "departments" else "employee_id"
    t2_col   = "manager_id" if t2 == "departments" else "customer_id"
    jt = random.choice(["INNER JOIN", "LEFT JOIN", "RIGHT JOIN"])
    templates = [
        (f"Get employee names and their {t2} info using a join",
         f"SELECT e.name, d.* FROM {t1} e {jt} {t2} d ON e.{join_col} = d.{t2_col};"),
        (f"Join {t1} and {t2} on {join_col}",
         f"SELECT * FROM {t1} {jt} {t2} ON {t1}.{join_col} = {t2}.{t2_col};"),
        (f"Combine {t1} with {t2} to show related data",
         f"SELECT {t1}.name, {t2}.* FROM {t1} {jt} {t2} ON {t1}.{join_col} = {t2}.{t2_col};")
    ]
    return random.choice(templates)


def sql_alter(_):
    t = rnd_table()
    new_col = random.choice(["phone_number VARCHAR(20)", "address TEXT", "age INT", "notes TEXT", "is_deleted BOOLEAN"])
    new_col_name = new_col.split()[0]
    templates = [
        (f"Add a {new_col_name} column to the {t} table",
         f"ALTER TABLE {t} ADD COLUMN {new_col};"),
        (f"Modify the {t} table to include {new_col_name}",
         f"ALTER TABLE {t} ADD {new_col};"),
        (f"Drop column status from {t}",
         f"ALTER TABLE {t} DROP COLUMN status;")
    ]
    return random.choice(templates)


def sql_drop(_):
    t = rnd_table()
    templates = [
        (f"Drop the {t} table",               f"DROP TABLE {t};"),
        (f"Delete the entire {t} table from the database", f"DROP TABLE IF EXISTS {t};"),
        (f"Remove {t} table permanently",      f"DROP TABLE {t};")
    ]
    return random.choice(templates)


def sql_view(_):
    t = rnd_table()
    view_name = f"v_{t}_summary"
    fields = rnd_fields(t, 3)
    templates = [
        (f"Create a view showing {', '.join(fields)} from {t}",
         f"CREATE VIEW {view_name} AS SELECT {', '.join(fields)} FROM {t};"),
        (f"Make a view called {view_name} for {t}",
         f"CREATE OR REPLACE VIEW {view_name} AS SELECT * FROM {t};"),
        (f"Build a summary view of {t}",
         f"CREATE VIEW {view_name} AS SELECT {', '.join(fields)} FROM {t} WHERE status = 'active';")
    ]
    return random.choice(templates)


def sql_use(_):
    db = random.choice(DB_NAMES)
    templates = [
        (f"Switch to the {db} database",              f"USE {db};"),
        (f"Connect to {db}",                          f"USE {db};"),
        (f"Select the {db} database for queries",     f"USE {db};")
    ]
    return random.choice(templates)


SQL_GENERATORS = {
    "create": sql_create,
    "read":   sql_read,
    "update": sql_update,
    "delete": sql_delete,
    "join":   sql_join,
    "alter":  sql_alter,
    "drop":   sql_drop,
    "view":   sql_view,
    "use":    sql_use
}
print("✅ SQL generators ready")

✅ SQL generators ready


In [6]:
def mongo_create(_):
    t = rnd_table()
    fields = SCHEMAS[t]["fields"]
    doc = {f: rnd_val(f) for f in fields}
    templates = [
        (f"Insert a new document into {t} collection",
         f"db.{t}.insertOne({json.dumps(doc)});"),
        (f"Add a record to the {t} collection with sample data",
         f"db.{t}.insertOne({json.dumps(doc)});"),
        (f"Create a new {t} entry",
         f"db.{t}.insertOne({json.dumps(doc)});")
    ]
    return random.choice(templates)


def mongo_read(_):
    t = rnd_table()
    f = rnd_field(t)
    v = rnd_val(f)
    templates = [
        (f"Find all documents in {t} where {f} is {v}",
         f'db.{t}.find({{"{f}": {json.dumps(v)}}});'),
        (f"Get all records from {t} collection",
         f"db.{t}.find({{}});"),
        (f"Retrieve one document from {t} where {f} equals {v}",
         f'db.{t}.findOne({{"{f}": {json.dumps(v)}}});'),
        (f"Show first 10 documents from {t}",
         f"db.{t}.find({{}}).limit(10);"),
        (f"Count documents in {t} collection",
         f"db.{t}.countDocuments({{}});")
    ]
    return random.choice(templates)


def mongo_update(_):
    t = rnd_table()
    filter_f = rnd_field(t)
    filter_v = rnd_val(filter_f)
    set_f = rnd_field(t)
    set_v = rnd_val(set_f)
    templates = [
        (f"Update {set_f} to {set_v} in {t} where {filter_f} is {filter_v}",
         f'db.{t}.updateOne({{"{filter_f}": {json.dumps(filter_v)}}}, {{"$set": {{"{set_f}": {json.dumps(set_v)}}}}});'),
        (f"Set {set_f} field to {set_v} for matching {t} documents",
         f'db.{t}.updateMany({{"{filter_f}": {json.dumps(filter_v)}}}, {{"$set": {{"{set_f}": {json.dumps(set_v)}}}}});'),
        (f"Modify {set_f} in {t} collection",
         f'db.{t}.updateOne({{"{filter_f}": {json.dumps(filter_v)}}}, {{"$set": {{"{set_f}": {json.dumps(set_v)}}}}});')
    ]
    return random.choice(templates)


def mongo_delete(_):
    t = rnd_table()
    f = rnd_field(t)
    v = rnd_val(f)
    templates = [
        (f"Delete documents from {t} where {f} is {v}",
         f'db.{t}.deleteOne({{"{f}": {json.dumps(v)}}});'),
        (f"Remove all {t} records where {f} equals {v}",
         f'db.{t}.deleteMany({{"{f}": {json.dumps(v)}}});'),
        (f"Erase {t} document with {f} = {v}",
         f'db.{t}.deleteOne({{"{f}": {json.dumps(v)}}});')
    ]
    return random.choice(templates)


def mongo_join(_):
    t1 = "orders"
    t2 = random.choice(["customers", "products"])
    local   = "customer_id" if t2 == "customers" else "product_id"
    foreign = "customer_id" if t2 == "customers" else "product_id"
    templates = [
        (f"Join {t1} with {t2} using lookup",
         f'db.{t1}.aggregate([{{"$lookup": {{"from": "{t2}", "localField": "{local}", "foreignField": "{foreign}", "as": "{t2}_info"}}}}]);'),
        (f"Get {t1} with {t2} details using aggregation",
         f'db.{t1}.aggregate([{{"$lookup": {{"from": "{t2}", "localField": "{local}", "foreignField": "{foreign}", "as": "{t2}_data"}}}}]);')
    ]
    return random.choice(templates)


def mongo_alter(_):
    t = rnd_table()
    new_field = random.choice(["phone_number", "address", "age", "notes", "is_deleted"])
    new_val   = random.choice(["N/A", "", 0, False])
    templates = [
        (f"Add {new_field} field to all {t} documents",
         f'db.{t}.updateMany({{}}, {{"$set": {{"{new_field}": {json.dumps(new_val)}}}}});'),
        (f"Rename a field in {t} collection",
         f'db.{t}.updateMany({{}}, {{"$rename": {{"old_field": "new_field"}}}});')
    ]
    return random.choice(templates)


def mongo_drop(_):
    t = rnd_table()
    templates = [
        (f"Drop the {t} collection",                       f"db.{t}.drop();"),
        (f"Delete the entire {t} collection from MongoDB", f"db.{t}.drop();"),
        (f"Remove {t} collection permanently",             f"db.{t}.drop();")
    ]
    return random.choice(templates)


def mongo_view(_):
    t = rnd_table()
    view_name = f"{t}_summary"
    f = rnd_field(t)
    templates = [
        (f"Create a view of {t} grouped by {f}",
         f'db.createView("{view_name}", "{t}", [{{"$group": {{"_id": "${f}", "count": {{"$sum": 1}}}}}}]);'),
        (f"Make a summary view for {t}",
         f'db.createView("{view_name}", "{t}", [{{"$project": {{"_id": 0, "{f}": 1}}}}]);')
    ]
    return random.choice(templates)


def mongo_use(_):
    db = random.choice(DB_NAMES)
    templates = [
        (f"Switch to the {db} database in MongoDB", f"use {db};"),
        (f"Connect to {db} in MongoDB shell",        f"use {db};"),
        (f"Select {db} as the active database",      f"use {db};")
    ]
    return random.choice(templates)


MONGO_GENERATORS = {
    "create": mongo_create,
    "read":   mongo_read,
    "update": mongo_update,
    "delete": mongo_delete,
    "join":   mongo_join,
    "alter":  mongo_alter,
    "drop":   mongo_drop,
    "view":   mongo_view,
    "use":    mongo_use
}
print("✅ MongoDB generators ready")

✅ MongoDB generators ready


In [7]:
OPERATIONS       = ["create", "read", "update", "delete", "join", "alter", "drop", "view", "use"]
SAMPLES_PER_OP   = 1000   # 9000 total per language

all_records = []

for lang, generators in [("SQL", SQL_GENERATORS), ("MongoDB", MONGO_GENERATORS)]:
    print(f"\n{'='*40}")
    print(f"  Generating {lang} samples...")
    print(f"{'='*40}")
    for op in OPERATIONS:
        op_records = []
        gen_fn = generators[op]
        for i in range(SAMPLES_PER_OP):
            nl, query = gen_fn(i)
            record = {
                "id":        f"{lang}_{op}_{i:04d}",
                "language":  lang,
                "operation": op,
                "input":     nl,
                "output":    query
            }
            op_records.append(record)
            all_records.append(record)

        # Per-operation split
        op_df = pd.DataFrame(op_records)
        op_train, op_test = train_test_split(op_df, test_size=0.3, random_state=42)
        op_train.to_csv(f"dataset/by_operation/{lang}_{op}_train.csv", index=False)
        op_test.to_csv( f"dataset/by_operation/{lang}_{op}_test.csv",  index=False)
        print(f"  ✅ {lang:8s} | {op:8s} → {len(op_train)} train / {len(op_test)} test")

print(f"\n📊 Total records: {len(all_records)}")


  Generating SQL samples...
  ✅ SQL      | create   → 700 train / 300 test
  ✅ SQL      | read     → 700 train / 300 test
  ✅ SQL      | update   → 700 train / 300 test
  ✅ SQL      | delete   → 700 train / 300 test
  ✅ SQL      | join     → 700 train / 300 test
  ✅ SQL      | alter    → 700 train / 300 test
  ✅ SQL      | drop     → 700 train / 300 test
  ✅ SQL      | view     → 700 train / 300 test
  ✅ SQL      | use      → 700 train / 300 test

  Generating MongoDB samples...
  ✅ MongoDB  | create   → 700 train / 300 test
  ✅ MongoDB  | read     → 700 train / 300 test
  ✅ MongoDB  | update   → 700 train / 300 test
  ✅ MongoDB  | delete   → 700 train / 300 test
  ✅ MongoDB  | join     → 700 train / 300 test
  ✅ MongoDB  | alter    → 700 train / 300 test
  ✅ MongoDB  | drop     → 700 train / 300 test
  ✅ MongoDB  | view     → 700 train / 300 test
  ✅ MongoDB  | use      → 700 train / 300 test

📊 Total records: 18000


In [8]:
df = pd.DataFrame(all_records)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Stratified split
strat_key = df["language"] + "_" + df["operation"]
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=strat_key)

# Save full + combined splits
df.to_csv("dataset/full_dataset.csv", index=False)
train_df.to_csv("dataset/train.csv",  index=False)
test_df.to_csv( "dataset/test.csv",   index=False)

# Save per-language splits
for lang in ["SQL", "MongoDB"]:
    lang_df = df[df["language"] == lang]
    l_train, l_test = train_test_split(lang_df, test_size=0.3, random_state=42)
    l_train.to_csv(f"dataset/{lang}_train.csv", index=False)
    l_test.to_csv( f"dataset/{lang}_test.csv",  index=False)

print("=" * 55)
print("  DATASET SUMMARY")
print("=" * 55)
print(f"  Total samples   : {len(df)}")
print(f"  Train samples   : {len(train_df)}  ({len(train_df)/len(df)*100:.0f}%)")
print(f"  Test  samples   : {len(test_df)}   ({len(test_df)/len(df)*100:.0f}%)")
print(f"  Languages       : {df['language'].nunique()}  ({', '.join(df['language'].unique())})")
print(f"  Operations      : {df['operation'].nunique()}")
print("\n  Per Language × Operation:")
print(df.groupby(["language","operation"])["id"].count().to_string())

print("\n📁 Files saved:")
for fname in sorted(os.listdir("dataset")):
    fpath = f"dataset/{fname}"
    if os.path.isfile(fpath):
        size = os.path.getsize(fpath) // 1024
        print(f"   {fname:40s}  {size:>5} KB")

print("\n✅ Data generation complete!")

  DATASET SUMMARY
  Total samples   : 18000
  Train samples   : 12600  (70%)
  Test  samples   : 5400   (30%)
  Languages       : 2  (SQL, MongoDB)
  Operations      : 9

  Per Language × Operation:
language  operation
MongoDB   alter        1000
          create       1000
          delete       1000
          drop         1000
          join         1000
          read         1000
          update       1000
          use          1000
          view         1000
SQL       alter        1000
          create       1000
          delete       1000
          drop         1000
          join         1000
          read         1000
          update       1000
          use          1000
          view         1000

📁 Files saved:
   MongoDB_test.csv                            435 KB
   MongoDB_train.csv                          1001 KB
   SQL_test.csv                                367 KB
   SQL_train.csv                               862 KB
   full_dataset.csv                          

In [9]:
!pip install transformers datasets sentencepiece sacrebleu accelerate -q
print("✅ Dependencies installed")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 2.8 MB/s eta 0:00:00
✅ Dependencies installed


In [10]:
import os, json, torch
import pandas as pd
import numpy as np
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer,
    Seq2SeqTrainer, Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import Dataset
import sacrebleu

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Device: {DEVICE}")
if DEVICE == "cuda":
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
else:
    print("   ⚠️ No GPU found — training will be slow! Enable T4 GPU in Runtime settings.")

✅ Device: cuda
   GPU: Tesla T4


In [11]:
CFG = {
    "model_name":     "t5-small",
    "max_input_len":  96,    # ✅ reduced from 128 (our prompts are short)
    "max_output_len": 192,   # ✅ reduced from 256 (queries are rarely that long)

    "epochs":         4,     # ✅ reduced from 10
    "batch_size":     32,    # ✅ increased from 8 (faster iterations)
    "learning_rate":  4e-4,
    "warmup_steps":   300,
    "weight_decay":   0.01,
    "eval_steps":     300,
    "save_steps":     300,

    "train_path":     "dataset/train.csv",
    "test_path":      "dataset/test.csv",
    "output_dir":     "./nl2query_model",
    "drive_save":     False,
    "drive_path":     "/content/drive/MyDrive/NL2Query/model"
}

os.makedirs(CFG["output_dir"], exist_ok=True)
print("✅ Config loaded")
for k, v in CFG.items():
    print(f"   {k:20s}: {v}")

✅ Config loaded
   model_name          : t5-small
   max_input_len       : 96
   max_output_len      : 192
   epochs              : 4
   batch_size          : 32
   learning_rate       : 0.0004
   warmup_steps        : 300
   weight_decay        : 0.01
   eval_steps          : 300
   save_steps          : 300
   train_path          : dataset/train.csv
   test_path           : dataset/test.csv
   output_dir          : ./nl2query_model
   drive_save          : False
   drive_path          : /content/drive/MyDrive/NL2Query/model


In [12]:
train_df = pd.read_csv(CFG["train_path"])
test_df  = pd.read_csv(CFG["test_path"])

print(f"✅ Train : {len(train_df)} rows")
print(f"   Test  : {len(test_df)} rows")
print(f"\n  Language distribution (train):")
print(train_df["language"].value_counts().to_string())
print(f"\n  Operation distribution (train):")
print(train_df["operation"].value_counts().to_string())
print("\n📋 Sample rows:")
print(train_df[["language","operation","input","output"]].head(4).to_string())

✅ Train : 12600 rows
   Test  : 5400 rows

  Language distribution (train):
language
MongoDB    6300
SQL        6300

  Operation distribution (train):
operation
delete    1400
read      1400
drop      1400
use       1400
update    1400
join      1400
alter     1400
create    1400
view      1400

📋 Sample rows:
  language operation                                                            input                                                  output
0  MongoDB    delete  Delete documents from employees where email is user@example.com  db.employees.deleteOne({"email": "user@example.com"});
1  MongoDB      read  Retrieve one document from products where product_id equals 991               db.products.findOne({"product_id": 991});
2  MongoDB      drop               Delete the entire products collection from MongoDB                                     db.products.drop();
3  MongoDB       use                   Switch to the inventory_db database in MongoDB                                  

In [13]:
tokenizer = T5Tokenizer.from_pretrained(CFG["model_name"])
model     = T5ForConditionalGeneration.from_pretrained(CFG["model_name"])
model     = model.to(DEVICE)

total_params = sum(p.numel() for p in model.parameters())
print(f"✅ Model  : {CFG['model_name']}  ({total_params/1e6:.1f}M parameters)")
print(f"   Device : {DEVICE}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/131 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

✅ Model  : t5-small  (60.5M parameters)
   Device : cuda


In [14]:
def build_prompt(row):
    """Structured prompt: language + operation give the model strong context."""
    return f"translate English to {row['language']} {row['operation']}: {row['input']}"

# Add prompt column
train_df["input_text"] = train_df.apply(build_prompt, axis=1)
test_df["input_text"]  = test_df.apply(build_prompt, axis=1)

print("📝 Example prompt:")
print("  ", train_df["input_text"].iloc[0])
print("📝 Expected output:")
print("  ", train_df["output"].iloc[0])


def tokenize_batch(batch):
    # Tokenize inputs
    model_inputs = tokenizer(
        batch["input_text"],
        max_length=CFG["max_input_len"],
        padding="max_length",
        truncation=True
    )

    # ✅ Fixed: tokenize labels directly (no as_target_tokenizer needed in modern Transformers)
    labels = tokenizer(
        text_target=batch["output"],      # <-- use text_target= instead of as_target_tokenizer
        max_length=CFG["max_output_len"],
        padding="max_length",
        truncation=True
    )

    label_ids = labels["input_ids"]
    # Replace padding token id with -100 so it's ignored in loss
    label_ids = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in label_ids
    ]
    model_inputs["labels"] = label_ids
    return model_inputs


# Convert to HuggingFace Dataset and tokenize
train_hf = Dataset.from_pandas(train_df[["input_text", "output"]].reset_index(drop=True))
test_hf  = Dataset.from_pandas(test_df[["input_text", "output"]].reset_index(drop=True))

print("\n⏳ Tokenizing train set...")
train_tok = train_hf.map(tokenize_batch, batched=True, batch_size=256,
                          remove_columns=["input_text", "output"])

print("⏳ Tokenizing test set...")
test_tok  = test_hf.map(tokenize_batch, batched=True, batch_size=256,
                         remove_columns=["input_text", "output"])

train_tok.set_format("torch")
test_tok.set_format("torch")

print(f"\n✅ Tokenization complete")
print(f"   Train : {len(train_tok)} samples")
print(f"   Test  : {len(test_tok)} samples")

📝 Example prompt:
   translate English to MongoDB delete: Delete documents from employees where email is user@example.com
📝 Expected output:
   db.employees.deleteOne({"email": "user@example.com"});

⏳ Tokenizing train set...


Map:   0%|          | 0/12600 [00:00<?, ? examples/s]

⏳ Tokenizing test set...


Map:   0%|          | 0/5400 [00:00<?, ? examples/s]


✅ Tokenization complete
   Train : 12600 samples
   Test  : 5400 samples


In [15]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    # Clip invalid token ids (can happen with beam search)
    preds = np.clip(preds, 0, tokenizer.vocab_size - 1)

    decoded_preds  = tokenizer.batch_decode(preds,  skip_special_tokens=True)
    labels         = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds  = [p.strip() for p in decoded_preds]
    decoded_labels = [l.strip() for l in decoded_labels]

    # BLEU
    bleu = sacrebleu.corpus_bleu(decoded_preds, [decoded_labels])

    # Exact match
    exact = sum(p == l for p, l in zip(decoded_preds, decoded_labels))
    exact_pct = exact / len(decoded_preds) * 100

    return {
        "bleu":        round(bleu.score, 2),
        "exact_match": round(exact_pct,  2)
    }

print("✅ Metrics function ready (BLEU + Exact Match)")

✅ Metrics function ready (BLEU + Exact Match)


In [16]:
training_args = Seq2SeqTrainingArguments(
    output_dir                  = CFG["output_dir"],
    num_train_epochs            = CFG["epochs"],
    per_device_train_batch_size = CFG["batch_size"],
    per_device_eval_batch_size  = CFG["batch_size"],
    warmup_steps                = CFG["warmup_steps"],
    weight_decay                = CFG["weight_decay"],
    learning_rate               = CFG["learning_rate"],
    eval_strategy         = "steps",
    eval_steps                  = CFG["eval_steps"],
    save_strategy               = "steps",
    save_steps                  = CFG["save_steps"],
    load_best_model_at_end      = True,
    metric_for_best_model       = "bleu",
    greater_is_better           = True,
    predict_with_generate       = True,
    generation_max_length       = CFG["max_output_len"],
    logging_steps               = 100,
    logging_dir     = "./logs",   # ✅ replaces deprecated logging_dir
    report_to                   = "none",
    fp16                        = (DEVICE == "cuda"),
)

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding=True,
    pad_to_multiple_of=8
)

# ✅ Fixed: 'tokenizer' argument removed from Seq2SeqTrainer in newer Transformers
# Use 'processing_class' instead, or just remove it (tokenizer is already in data_collator)
trainer = Seq2SeqTrainer(
    model           = model,
    args            = training_args,
    train_dataset   = train_tok,
    eval_dataset    = test_tok,
    data_collator   = data_collator,
    compute_metrics = compute_metrics,
    processing_class = tokenizer,        # ✅ replaces deprecated 'tokenizer=' argument
)

steps_per_epoch = len(train_tok) // CFG["batch_size"]
print("✅ Trainer ready")
print(f"   Steps per epoch : {steps_per_epoch}")
print(f"   Total steps     : {CFG['epochs'] * steps_per_epoch}")
print(f"   Eval every      : {CFG['eval_steps']} steps")

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


✅ Trainer ready
   Steps per epoch : 393
   Total steps     : 1572
   Eval every      : 300 steps


In [17]:
print("🚀 Starting training...\n")
train_result = trainer.train()

print("\n" + "="*50)
print("  ✅ TRAINING COMPLETE")
print("="*50)
print(f"  Training loss : {train_result.training_loss:.4f}")
print(f"  Runtime       : {train_result.metrics.get('train_runtime', 0):.0f}s")
print(f"  Samples/sec   : {train_result.metrics.get('train_samples_per_second', 0):.1f}")

🚀 Starting training...



/usr/local/lib/python3.12/dist-packages/transformers/data/data_collator.py:600: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss,Validation Loss,Bleu,Exact Match
300,0.299276,0.116278,78.550000,71.020000
600,0.102675,0.074897,93.230000,78.760000
900,0.082126,0.070292,93.280000,78.980000
1200,0.085580,0.068634,93.290000,79.350000
1500,0.079634,0.067733,93.270000,79.150000


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].



  ✅ TRAINING COMPLETE
  Training loss : 0.3574
  Runtime       : 1706s
  Samples/sec   : 29.5


In [18]:
print("⏳ Running evaluation on test set...")
eval_results = trainer.evaluate()

print("\n" + "="*50)
print("  📊 EVALUATION RESULTS")
print("="*50)
for k, v in eval_results.items():
    print(f"  {k:35s}: {v}")

⏳ Running evaluation on test set...



  📊 EVALUATION RESULTS
  eval_loss                          : 0.06863422691822052
  eval_bleu                          : 93.29
  eval_exact_match                   : 79.35
  eval_runtime                       : 231.6864
  eval_samples_per_second            : 23.307
  eval_steps_per_second              : 0.729
  epoch                              : 4.0


In [19]:
model.eval()

def generate_query(nl_input, language, operation, num_beams=4):
    prompt = f"translate English to {language} {operation}: {nl_input}"
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        max_length=CFG["max_input_len"],
        truncation=True
    ).to(DEVICE)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=CFG["max_output_len"],
            num_beams=num_beams,
            early_stopping=True
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


print("📊 Per-Operation Exact Match Accuracy (on 50 samples each)")
print("=" * 60)

results_per_op = {}
OPERATIONS = ["create","read","update","delete","join","alter","drop","view","use"]

for lang in ["SQL", "MongoDB"]:
    print(f"\n  {lang}")
    print(f"  {'-'*40}")
    for op in OPERATIONS:
        subset = test_df[(test_df["language"] == lang) & (test_df["operation"] == op)].head(50)
        correct = 0
        for _, row in subset.iterrows():
            pred = generate_query(row["input"], lang, op)
            if pred.strip().lower() == row["output"].strip().lower():
                correct += 1
        acc = correct / len(subset) * 100
        results_per_op[f"{lang}_{op}"] = acc
        bar = "█" * int(acc / 5)
        print(f"  {op:8s}  {acc:5.1f}%  {bar}")

pd.DataFrame.from_dict(results_per_op, orient="index", columns=["exact_match_%"])\
  .to_csv("evaluation_results.csv")
print("\n✅ Saved to evaluation_results.csv")

📊 Per-Operation Exact Match Accuracy (on 50 samples each)

  SQL
  ----------------------------------------
  create     96.0%  ███████████████████
  read       88.0%  █████████████████
  update     24.0%  ████
  delete     74.0%  ██████████████
  join       44.0%  ████████
  alter     100.0%  ████████████████████
  drop      100.0%  ████████████████████
  view       78.0%  ███████████████
  use       100.0%  ████████████████████

  MongoDB
  ----------------------------------------
  create      0.0%  
  read        0.0%  
  update      0.0%  
  delete      0.0%  
  join        0.0%  
  alter       0.0%  
  drop      100.0%  ████████████████████
  view        0.0%  
  use       100.0%  ████████████████████

✅ Saved to evaluation_results.csv


In [20]:
# Save model + tokenizer
model.save_pretrained(CFG["output_dir"])
tokenizer.save_pretrained(CFG["output_dir"])

# Save metadata (used by app.py)
meta = {
    "model_name":    CFG["model_name"],
    "languages":     ["SQL", "MongoDB"],
    "operations":    ["create","read","update","delete","join","alter","drop","view","use"],
    "max_input_len": CFG["max_input_len"],
    "max_output_len":CFG["max_output_len"],
    "prompt_format": "translate English to {language} {operation}: {nl_query}"
}
with open(f"{CFG['output_dir']}/model_meta.json", "w") as f:
    json.dump(meta, f, indent=2)

print(f"✅ Model saved to: {CFG['output_dir']}")
print(f"   Files: {os.listdir(CFG['output_dir'])}")

# Optional: save to Google Drive
if CFG["drive_save"]:
    from google.colab import drive
    import shutil
    drive.mount("/content/drive")
    shutil.copytree(CFG["output_dir"], CFG["drive_path"], dirs_exist_ok=True)
    print(f"✅ Also saved to Drive: {CFG['drive_path']}")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Model saved to: ./nl2query_model
   Files: ['model_meta.json', 'checkpoint-1576', 'checkpoint-900', 'config.json', 'checkpoint-300', 'tokenizer.json', 'checkpoint-1500', 'checkpoint-1200', 'generation_config.json', 'tokenizer_config.json', 'checkpoint-600', 'model.safetensors']


In [21]:
test_cases = [
    ("Show all employees in the HR department",           "SQL",     "read"),
    ("Insert a new product into products",                "SQL",     "create"),
    ("Update salary to 75000 where employee_id is 5",    "SQL",     "update"),
    ("Delete all orders where status is cancelled",       "SQL",     "delete"),
    ("Join employees with departments on manager_id",     "SQL",     "join"),
    ("Add a phone_number column to customers",            "SQL",     "alter"),
    ("Drop the inventory table",                          "SQL",     "drop"),
    ("Create a view of active employees",                 "SQL",     "view"),
    ("Switch to the hr_db database",                      "SQL",     "use"),
    ("Find all orders where status is pending",           "MongoDB", "read"),
    ("Insert a new customer document",                    "MongoDB", "create"),
    ("Update salary for employee 101 to 80000",          "MongoDB", "update"),
    ("Delete customers from Mumbai",                      "MongoDB", "delete"),
    ("Join orders with customers using lookup",           "MongoDB", "join"),
    ("Drop the inventory collection",                     "MongoDB", "drop"),
]

print("🎯 INFERENCE DEMO")
print("=" * 80)
for nl, lang, op in test_cases:
    pred = generate_query(nl, lang, op)
    print(f"\n  NL    : {nl}")
    print(f"  Lang  : {lang}  |  Op: {op}")
    print(f"  Query : {pred}")

🎯 INFERENCE DEMO

  NL    : Show all employees in the HR department
  Lang  : SQL  |  Op: read
  Query : SELECT * FROM employees INNER DISTINCT;

  NL    : Insert a new product into products
  Lang  : SQL  |  Op: create
  Query : CREATE products (product_id INT, name VARCHAR(100), category VARCHAR(50), price DECIMAL(10,2), stock INT, supplier_id INT, created_at DATETIME);

  NL    : Update salary to 75000 where employee_id is 5
  Lang  : SQL  |  Op: update
  Query : UPDATE salary SET salary = 75000 WHERE employee_id = 5;

  NL    : Delete all orders where status is cancelled
  Lang  : SQL  |  Op: delete
  Query : DELETE FROM orders WHERE status = 'cancelled';

  NL    : Join employees with departments on manager_id
  Lang  : SQL  |  Op: join
  Query : SELECT * FROM employees INNER JOIN departments ON employees.manager_id = departments.manager_id;

  NL    : Add a phone_number column to customers
  Lang  : SQL  |  Op: alter
  Query : ALTER TABLE customers ADD COLUMN phone_number VARCHAR

In [22]:
import shutil
import os
from google.colab import files

# ✅ Zip only the essential model files, skipping checkpoint folders
ZIP_NAME   = "nl2query_model"
MODEL_DIR  = CFG["output_dir"]
TEMP_DIR   = "./nl2query_model_export"

# Clean up temp dir if it exists
if os.path.exists(TEMP_DIR):
    shutil.rmtree(TEMP_DIR)
os.makedirs(TEMP_DIR)

# ✅ Copy only essential files (skip checkpoint-* folders)
ESSENTIAL_EXTENSIONS = [
    ".json",        # config, tokenizer config, model meta
    ".model",       # sentencepiece tokenizer model
    ".bin",         # pytorch model weights
    ".safetensors", # alternative weights format
    ".txt",         # special tokens, vocab
]

copied = []
for fname in os.listdir(MODEL_DIR):
    fpath = os.path.join(MODEL_DIR, fname)
    # Skip checkpoint directories
    if os.path.isdir(fpath):
        print(f"  ⏭️  Skipping folder : {fname}/")
        continue
    # Only copy essential file types
    ext = os.path.splitext(fname)[1].lower()
    if ext in ESSENTIAL_EXTENSIONS:
        shutil.copy2(fpath, os.path.join(TEMP_DIR, fname))
        size_kb = os.path.getsize(fpath) // 1024
        print(f"  ✅ Copied : {fname:45s} ({size_kb:>6} KB)")
        copied.append(fname)
    else:
        print(f"  ⏭️  Skipping file   : {fname}")

print(f"\n📦 Zipping {len(copied)} files...")
shutil.make_archive(ZIP_NAME, "zip", TEMP_DIR)

# Cleanup temp dir
shutil.rmtree(TEMP_DIR)

size_mb = os.path.getsize(f"{ZIP_NAME}.zip") / 1024 / 1024
print(f"✅ Zip created : {ZIP_NAME}.zip  ({size_mb:.1f} MB)")
print("⬇️  Downloading...")
files.download(f"{ZIP_NAME}.zip")
print("✅ Done! Unzip and point app.py MODEL_PATH to the folder.")

  ✅ Copied : model_meta.json                               (     0 KB)
  ⏭️  Skipping folder : checkpoint-1576/
  ⏭️  Skipping folder : checkpoint-900/
  ✅ Copied : config.json                                   (     1 KB)
  ⏭️  Skipping folder : checkpoint-300/
  ✅ Copied : tokenizer.json                                (  2057 KB)
  ⏭️  Skipping folder : checkpoint-1500/
  ⏭️  Skipping folder : checkpoint-1200/
  ✅ Copied : generation_config.json                        (     0 KB)
  ✅ Copied : tokenizer_config.json                         (     2 KB)
  ⏭️  Skipping folder : checkpoint-600/
  ✅ Copied : model.safetensors                             (236369 KB)

📦 Zipping 6 files...
✅ Zip created : nl2query_model.zip  (211.9 MB)
⬇️  Downloading...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Done! Unzip and point app.py MODEL_PATH to the folder.
